In [27]:
import pandas as pd
from embedding import Embedding
import torch.nn as nn
import torch
import numpy as np
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import time

In [28]:
data = pd.read_csv('C:/Users/user/OneDrive/문서/dev/UROP/Crawling/data/sample_comment.csv')
token = pd.read_csv('C:/Users/user/OneDrive/문서/dev/UROP/Crawling/tokenized_0.csv')

In [29]:
df = data[['mbti','comments']][:800]

In [30]:
for i in range(len(df)):
    if 'E' in df.loc[i, 'mbti']:
        df.loc[i, 'mbti'] = 0
    else:
        df.loc[i, 'mbti'] = 1

In [31]:
label = [int(i) for i in df['mbti'].values.tolist()]
print(len(label))
label_tensor = torch.LongTensor(label)
print(label_tensor)

800
tensor([0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0,
        0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0,
        1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1,
        1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0,
        1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [32]:
df = df.dropna() # 결측치 왠지모르게 있어서 제거함
token = token.dropna()

In [33]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.hidden = self.init_hidden()
        # self.embedding = nn.Embedding(vocab_size, input_size)

    def init_hidden(self):
        return (torch.zeros(1, 1, self.hidden_size),
                torch.zeros(1, 1, self.hidden_size))

    def forward(self, input):
        # embedded = self.embedding(input)
        lstm_out, _ = self.lstm(input)

        # print(input.size(-1))
        # lstm_out, self.hidden = self.lstm(input.view(len(input), 1, -1), self.hidden)
        output = self.fc(lstm_out[-1])
        return output

In [34]:
em = Embedding(sereis=token['comments'])
word2vec_model = em.get_embedding_model()

[I 2023-12-10 03:09:04,185] A new study created in memory with name: no-name-aac962eb-70bc-4408-b5be-bba2c91dccea
[I 2023-12-10 03:09:04,259] Trial 0 finished with value: 62974.18359375 and parameters: {'vector_size': 36, 'window': 8, 'min_count': 9, 'sg': 0}. Best is trial 0 with value: 62974.18359375.


In [35]:
# Word2Vec 모델의 크기를 기반으로 LSTM 모델 생성
input_size = word2vec_model.vector_size
hidden_size = 128
output_size = 1
# vocab_size = len(word2vec_model.wv.key_to_index)
lstm_model = LSTMModel(input_size, hidden_size, output_size)
# print(vocab_size)

In [36]:
input_size

36

In [37]:
# 단어에 대한 0 벡터 생성
def get_word_vector(word, model):
    try:
        return model.wv[word]
    except:
        return np.zeros(input_size)

# 모든 단어에 대한 벡터 리스트 생성
# word_vectors = [get_word_vector(word, word2vec_model) for sentence in em.corpus for word in sentence]
end = 100
word_vectors = []

for sentence in em.corpus:
    temp = []
    for word in sentence[:end]:  # 문장을 end 길이로 자르기
        temp.append(get_word_vector(word, word2vec_model))

    # 패딩 추가
    padding_length = end - len(temp)
    if padding_length > 0:
        padding = [np.zeros(word2vec_model.vector_size) for _ in range(padding_length)]
        temp.extend(padding)

    word_vectors.append(temp)

In [52]:
df[140:150]

,mbti,comments
141,1,솔직함 돌직구 확실하게 표현 이 삼박자가 잘 어우러진 사람 이라고 봅니다 근데 이제...
142,1,글을 적으심에 흡수력이 있으시네요 읽다보면 빠져들어요 술술 읽히고요
143,1,앗 좋게 봐주셔서 감사합니다
145,1,인프제 나는 함께하지 못한 15년의 시간을 알고 있잖아요 그때 누나는 얼마나 더 예...
146,1,성향차이가 뚜렷하네요 인프제가 계속 저러니 제가 더 소중히 대하려 하다가 가끔 고...
147,1,와 역대급
148,1,인프제 참 사랑스럽습니다
149,1,오 갑자기 제 소개팅남 인프제가 순간 생각난 순간이엿어 갑자기 확 스쳐지나가네요
150,1,잘 버리셨습니다 단호
151,1,다른 사랑 찾아 떠나는중입니다


In [38]:
# word_to_index = {word: index for index, word in enumerate(word2vec_model.wv.key_to_index)}

# 단어에 대한 0 벡터 생성
def get_word_vector(word, model):
    try:
        return model.wv[word]
    except:
        return np.zeros(input_size)


# 모든 단어에 대한 벡터 리스트 생성
# word_vectors = [get_word_vector(word, word2vec_model) for sentence in em.corpus for word in sentence]
end = 100
word_vectors = []

for sentence in em.corpus:
    temp = []
    for word in sentence[:end]:  # 문장을 end 길이로 자르기
        temp.append(get_word_vector(word, word2vec_model))

    # 패딩 추가
    padding_length = end - len(temp)
    if padding_length > 0:
        padding = [np.zeros(word2vec_model.vector_size) for _ in range(padding_length)]
        temp.extend(padding)

    word_vectors.append(temp)

In [39]:
print(word_vectors[0])
print(len(word_vectors[0]))

[array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.]), array([-0.22162798,  0.1405233 , -0.48672017, -0.060675  , -0.55717903,
        0.54147846, -0.4732247 ,  0.44832304, -0.5338283 , -0.4358954 ,
        0.45082185,  0.49487755,  0.38412744, -0.3684534 , -0.7386251 ,
        0.14234127,  0.06680071,  0.05499512,  0.87546295,  0.24309716,
        0.12383009, -0.44432822,  1.358831  ,  0.7081141 , -0.1322447 ,
       -0.7458802 , -0.10322662, -0.31236213, -0.20551728, -0.4976575 ,
       -0.6453013 , -0.06790318,  0.6550766 ,  0.12245926,  1.1814091 ,
       -0.41208228], dtype=float32), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.]), array([-0.34390262,  0.2365982 , -0.7737078 , -0.11658251, -0.8171447 ,
        0.85993075, -0.71419775,  0.69681674, -0.867

In [40]:
for i in word_vectors:
    if len(i) != 100:
        print(i)

In [41]:
# 벡터 리스트를 텐서로 변환
input_tensor = torch.tensor(word_vectors, dtype=torch.float32)
# input_tensor = torch.LongTensor(word_vectors)

In [42]:
trn_data=input_tensor[:800]
trn_label=label_tensor[:800]
val_data=input_tensor[800:]
val_label=label_tensor[800:]

In [43]:
input_tensor

tensor([[[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [-0.2216,  0.1405, -0.4867,  ...,  0.1225,  1.1814, -0.4121],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.3167,  0.1640, -0.6284,  ...,  0.1582,  1.4594, -0.5666],
         [-0.3542,  0.2180, -0.6552,  ...,  0.1934,  1.6517, -0.6209],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],

        [[-0.1121,  0.0869, -0.1852,  ...,  0.0720,  0.4887, -0.1720],
         [-0.2021,  0.1585, -0.4829,  ...,  0

In [44]:
# len(input_tensor[0])
print(input_tensor.shape)
print(input_tensor.size(-1))

torch.Size([987, 100, 36])
36


In [45]:
# LSTM 모델에 입력 전달하여 출력 받기
output = lstm_model(input_tensor)
print("LSTM 모델의 출력:", output, len(output))

LSTM 모델의 출력: tensor([[0.0977],
        [0.0966],
        [0.0979],
        [0.0981],
        [0.0947],
        [0.0974],
        [0.0986],
        [0.0985],
        [0.0951],
        [0.1019],
        [0.1085],
        [0.1082],
        [0.1021],
        [0.0988],
        [0.1001],
        [0.1031],
        [0.1019],
        [0.1033],
        [0.1060],
        [0.1106],
        [0.1121],
        [0.1081],
        [0.0995],
        [0.0988],
        [0.0944],
        [0.0977],
        [0.0988],
        [0.1021],
        [0.0993],
        [0.1006],
        [0.1029],
        [0.1036],
        [0.0990],
        [0.0975],
        [0.1033],
        [0.1028],
        [0.1031],
        [0.1025],
        [0.1030],
        [0.1036],
        [0.1019],
        [0.1012],
        [0.1020],
        [0.1018],
        [0.1061],
        [0.1069],
        [0.1028],
        [0.1039],
        [0.1075],
        [0.1098],
        [0.1120],
        [0.1106],
        [0.1045],
        [0.1030],
        [0.1017

In [46]:
# BCE 손실 함수 정의
criterion = nn.BCEWithLogitsLoss()
# Adam 옵티마이저 설정
optimizer = optim.Adam(lstm_model.parameters(), lr=0.001)

In [47]:
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

In [48]:
# data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
# dataset = MyDataset(input_tensor, label_tensor)
trn_dataset = MyDataset(trn_data, trn_label)
val_dataset = MyDataset(val_data, val_label)

batch_size = 100
train_loader = DataLoader (trn_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader (val_dataset, batch_size=batch_size, shuffle=True)

In [49]:
# 학습 및 평가 루프 정의
def train(model, criterion, optimizer, train_loader, epochs=5):
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:  
            # train_loader에는 입력 데이터와 레이블이 있는 데이터가 포함되어야 합니다.
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())  
            # BCELoss를 사용하므로 레이블을 float로 변환합니다.
            
            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")

In [50]:
def evaluate(model, criterion, eval_loader):
    model.eval()  # 평가 모드로 모델 설정
    total_loss = 0.0
    total_corrects = 0
    with torch.no_grad():  # 기울기 계산 비활성화
        for inputs, labels in eval_loader:
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())

            total_loss += loss.item()
            # 예측된 클래스를 계산하고, 정확도를 측정합니다.
            # 이 부분은 사용하는 모델과 문제에 따라 다를 수 있습니다.
            preds = outputs.round()  # 예: 이진 분류의 경우
            total_corrects += torch.sum(preds == labels.data)

    # 평균 손실과 정확도 계산
    avg_loss = total_loss / len(eval_loader)
    accuracy = total_corrects.double() / len(eval_loader.dataset)

    print(f"Loss: {avg_loss}, Accuracy: {accuracy}")

    return avg_loss, accuracy

In [51]:
def train_and_evaluate(model, criterion, optimizer, train_loader, eval_loader, epochs=5):
    for epoch in range(epochs):
        print(f"Epoch {epoch+1}/{epochs}")

        # 학습 루프
        train(model, criterion, optimizer, train_loader, epochs=1)

        # 평가 루프
        eval_loss, eval_accuracy = evaluate(model, criterion, eval_loader)

        # 에포크 결과 출력
        print(f"Validation Loss: {eval_loss}, Validation Accuracy: {eval_accuracy}")

# 모델, 손실 함수, 옵티마이저, 데이터 로더 등을 정의한 후
train_and_evaluate(lstm_model, criterion, optimizer, train_loader, val_loader, epochs=5)


Epoch 1/5


Epoch 1/1, Loss: 0.6456194147467613


IndexError: index 146 is out of bounds for dimension 0 with size 0